In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from string import ascii_uppercase

video_folder = "videos"           # Folder with char_1.mp4 ... char_6.mp4
output_folder = "output_tracks"
os.makedirs(output_folder, exist_ok=True)

video_files = [f"char_{i}.mp4" for i in range(1, 7)]

# --- Auto-detect dominant car color ---
def get_car_color_range(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    pixels = hsv.reshape(-1, 3)
    median_color = np.median(pixels, axis=0)
    h, s, v = [int(x) for x in median_color]  # ✅ convert to int
    lower = np.array([max(h - 15, 0), max(s - 60, 50), max(v - 60, 50)], dtype=np.uint8)
    upper = np.array([min(h + 15, 179), 255, 255], dtype=np.uint8)
    return lower, upper

# --- Extract trajectory ---
def extract_trajectory(video_path, live_display=True):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        cap.release()
        print(f"⚠️ Could not read {video_path}")
        return []

    lower, upper = get_car_color_range(frame)
    positions = []
    trail = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower, upper)  # ✅ Fixed dtype issue
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            c = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(c)
            cx, cy = x + w // 2, y + h // 2
            positions.append((cx, cy))
            trail.append((cx, cy))
            if len(trail) > 30:
                trail.pop(0)

            if live_display:
                for t in range(1, len(trail)):
                    cv2.line(frame, trail[t-1], trail[t], (0, 255, 0), 2)
                cv2.circle(frame, (cx, cy), 6, (0, 0, 255), -1)
                cv2.imshow(f"Tracking {os.path.basename(video_path)}", frame)
                if cv2.waitKey(10) & 0xFF == 27:
                    break

    cap.release()
    if live_display:
        cv2.destroyAllWindows()
    return positions

# --- Convert trajectory to image ---
def trajectory_to_image(positions, size=128):
    img = np.zeros((size, size), dtype=np.uint8)
    if len(positions) < 2:
        return img
    xs, ys = zip(*positions)
    xs = np.interp(xs, (min(xs), max(xs)), (10, size - 10))
    ys = np.interp(ys, (min(ys), max(ys)), (10, size - 10))
    pts = np.array(list(zip(xs, ys)), np.int32)
    cv2.polylines(img, [pts], False, 255, 4)
    return img

# --- Create letter templates ---
def create_letter_templates(font=cv2.FONT_HERSHEY_SIMPLEX, size=128):
    templates = {}
    for ch in ascii_uppercase:
        img = np.zeros((size, size), dtype=np.uint8)
        cv2.putText(img, ch, (20, int(size * 0.75)), font, 3, 255, 8, cv2.LINE_AA)
        templates[ch] = img
    return templates

# --- Match trajectory to best letter ---
def match_letter(traj_img, templates):
    best_score = -1
    best_letter = "?"
    for ch, tmp in templates.items():
        score = cv2.matchTemplate(traj_img, tmp, cv2.TM_CCOEFF_NORMED).max()
        if score > best_score:
            best_score = score
            best_letter = ch
    return best_letter, best_score

# --- Main execution ---
templates = create_letter_templates()
flag_letters = []

for vid in video_files:
    path = os.path.join(video_folder, vid)
    if not os.path.exists(path):
        print(f"❌ Missing {path}")
        continue

    print(f"🚗 Processing {vid} ...")
    positions = extract_trajectory(path, live_display=True)
    traj_img = trajectory_to_image(positions)

    plt.imshow(traj_img, cmap='gray')
    plt.axis('off')
    plt.title(vid)
    plt.savefig(os.path.join(output_folder, vid.replace('.mp4', '.png')), bbox_inches='tight')
    plt.close()

    letter, score = match_letter(traj_img, templates)
    flag_letters.append(letter)
    print(f"✅ {vid}: → {letter} (score={score:.3f})")

flag = "FLAG{" + "".join(flag_letters) + "}"
print("\n🏁 Predicted FLAG:", flag)

🚗 Processing char_1.mp4 ...
